In [1]:
# Computes stochastic closure certificate using SOS for stochastic systems in verifying LTL specifications
# Numerical Example from [24]

# include important libraries
using JuMP
using MosekTools
using DynamicPolynomials
using MultivariatePolynomials
using LinearAlgebra
using TSSOS # important for SOS, see https://github.com/wangjie212/TSSOS
using Distributions # for the noise

In [2]:
@polyvar x1 x2 y1 y2 x01 x02 # global vars used in monomials
vars = [x1, x2, y1, y2]

sos_tol = 1 # the maximum degree of unknown SOS lagrangian polynomials = deg + sos_tol 
error = 2   # precision digit places
gamma = 0.07
lamda = 115
# p-sat = 99.939%
tau1, tau2, delta = .05, .05, 15 # S-procedure constants in condition 3

bd = 200 # state bound
# noise generator
function noise()
    return rand(Uniform(-1, 1))
end

# build expectation E[.|.] of system dynamics f_sym(x1, x2) symbolically
# f_symx1 = 0.5 * x1 * (1 + x2 + noise()) => E[f_symx1 | x1] = 0.5 * x1 * (1 + x2)
# f_symx2 = 0.5 * x2 => E[f_symx2 | x2] = f_symx2


# creates the parametrized SCC
function add_paramC_poly!(model, var1, deg, q, p)
    basis = monomials(var1, 0:deg) # basis in x and y
    coeffs = @variable(model, [1:length(basis)], base_name="c_$(q)_$(p)")
    C_qp = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return C_qp, coeffs, basis
end

# creates the parametrized ranking functions
function add_paramV_poly!(model, var2, deg, q, p)
    basis = monomials(var2, 0:deg)
    coeffs = @variable(model, [1:length(basis)], base_name="v_$(q)_$(p)")
    V_qp = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return V_qp, coeffs, basis
end

# the considered NBA 
# ~A representing ~(Fa) = G(~a)
S = Dict(
    :"q0" => [(Set(["b"]), :"q0"), (Set(["a"]), :"q1")],
    :"q1" => [(Set(["a", "b"]), :"q1")]
)

Q0, QF = Set(["q0"]), Set(["q0"]) # Initial and Acceptance set of ~NBA

eps = 1e-10 # for the strict inequality in describing poly domain

# considering that AP = {{a}, {b}}
g_a1, g_a2 = [x1-(0+eps), (1-eps)-x1, x2-(0+eps), (1-eps)-x2], [x1-(0+eps), (1-eps)-x1, x2-(0+eps), (1-eps)-x2, y1-(0+eps), (1-eps)-y1, y2-(0+eps), (1-eps)-y2] # X_a = (0, 1)^2
# X_b = X\X_a = [-bd, 0] U [1, bd]
g_b11, g_b21 = [x1+bd, 0-x1, x2+bd, 0-x2], [x1+bd, 0-x1, x2+bd, 0-x2, y1+bd, 0-y1, y2+bd, 0-y2] # [-bd, 0] 
g_b12, g_b22 = [x1-1, bd-x1, x2-1, bd-x2], [x1-1, bd-x1, x2-1, bd-x2, y1-1, bd-y1, y2-1, bd-y2] # [1, bd]

# for the first condition (x1, x2)
g11 = Dict(
    :"a" => g_a1, :"b" => g_b11
)
g12 = Dict(
    :"a" => g_a1, :"b" => g_b12
)
# for the 2nd condition
g21 = Dict(
    :"a" => g_a2, :"b" => g_b21
)  
g22 = Dict(
    :"a" => g_a2, :"b" => g_b22
)  

# g(x0,x,y) X = [-bd, bd]^2, X0 = {(2^i,2^i) | i=1,2,3,4,5}
g3 = [x01-2, 2-x01, x02-2, 2-x02, x01-4, 4-x01, x02-4, 4-x02, x01-8, 8-x01, x02-8, 8-x02, x01-16, 16-x01, x02-16, 16-x02, x01-32, 32-x01, x02-32, 32-x02,
        x1+bd, bd-x1, x2+bd, bd-x2, y1+bd, bd-y1, y2+bd, bd-y2]

# polynomial stochastic closure certificate of degree deg and NBA S
function scc(deg)
    # synthesize SCC by using the standard formulation
    # deg: degree of scc template
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)

    C_dict = Dict()  # key => (symbolic_poly, numeric_poly)
    
    for q in keys(S)
        for (lab_set, qn) in S[q]
            for lab in lab_set
                C, Cc, Cb = add_paramC_poly!(model, vars, deg, q, qn) # CC, CC coefficients, and CC basis functions
                # these modification takes care of the expectation
                EC = C(vars=>[x1,x2,0.5*x1*(1+x2),0.5*x2]) 
                # see TSSOS https://github.com/wangjie212/TSSOS
                key = (q, qn, lab)  
                C_dict[key] = (C, Cc, Cb)  # store symbolic, coefficients, basis

                # 1st condition
                add_psatz!(model, -EC + gamma, [x1, x2], g11[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                add_psatz!(model, -EC + gamma, [x1, x2], g12[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                for p in keys(S)
                    C1, Cc1, Cb1 = add_paramC_poly!(model, vars, deg, p, qn)
                    EC1 = C1(vars=>[x1,x2,0.5*y1*(1+y2),0.5*y2]) 
                    C2, Cc2, Cb2 = add_paramC_poly!(model, vars, deg, p, q)
                    key1, key2 = (p, qn), (p, q)  
                    C_dict[key1], C_dict[key2] = (C1, Cc1, Cb1), (C2, Cc2, Cb2) 
                    
                    # 2nd condition
                    add_psatz!(model, -EC1 + C2, vars, g21[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    add_psatz!(model, -EC1 + C2, vars, g22[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                end
            end
        end
    end
    for q0 in Q0
        for qf1 in QF
            V1, Vc1, Vb1 = add_paramV_poly!(model, [x01, x02, y1, y2], deg, q0, qf1)
            key3a = (q0, qf1, "V")  
            C_dict[key3a] = (V1, Vc1, Vb1)
            for qf2 in QF
                V2, Vc2, Vb2 = add_paramV_poly!(model, [x01, x02, x1, x2], deg, q0, qf2)
                C3, Cc3, Cb3 = add_paramC_poly!(model, [x01, x02, x1, x2], deg, q0, qf2)
                C4, Cc4, Cb4 = add_paramC_poly!(model, vars, deg, qf2, qf1)
                
                key3b = (q0, qf2, "V")  
                C_dict[key3b] = (V2, Vc2, Vb2)
                key4 = (q0, qf2)  
                C_dict[key4] = (C3, Cc3, Cb3)
                key5 = (qf1, qf2)  
                C_dict[key5] = (C4, Cc4, Cb4)
                # 3rd condition
                add_psatz!(model, -V1 + V2 - delta + tau1 * (lamda - C3) + tau2 * (lamda - C4), [x01, x02, x1, x2, y1, y2], g3, [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
            end
        end
    end
        
    optimize!(model) # solve for coefficients
    status = termination_status(model)
    C_eval_dict = Dict() # Get numerical values of coefficients and plug into polynomials
    for (key, (C, Cc, Cb)) in C_dict
        coeff_vals = round.(value.(Cc); digits=error)  # Round each coefficient to 2 decimal places
        C_numeric = sum(coeff_vals[i] * Cb[i] for i in eachindex(Cb))
        C_eval_dict[key] = (C, C_numeric)
    end
    
    # status might be optimal but if all Bc approx 10^{-error}, it's essentially 0 so OPTIMAL != CC.
    return status, C_eval_dict
end
        

# Simulation
file = open("./systems/prob_Reach_e.g_SCC_SOS.txt", "w")

deg = 1 # desired degree of SCC
stats = @timed (status, CC_data) = scc(deg)

write(file, "poly deg: "*string(deg)*"\n")
write(file, "status: "*string(status)*"\n")
write(file, "Number of SCC polynomials: "*string(length(CC_data))*"\n")
write(file, "time: "*string(stats.time)*"\n\n")

# Write the dictionary line-by-line
for (k, v) in CC_data
    write(file, "Key: $(k)\n")
    write(file, "Polynomial: $(v[1])\n")
    write(file, "Coefficients: $(v[2])\n\n")
end

close(file)

println("Finished")

Finished
